In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  


input1 = input('1. 크롤링할 키워드는 무엇입니까?(예:여행): ')

input2 = input('2. 결과에세 반드시 포함하는 단어를 입력하세요(예:국내,바닷가): ')

input3 = input('3. 결과에서 제외할 키워드를 입력하세요(예:분양권,해외): ')

input4 = input('4. 조회 시작일자 입력(예:2019-01-01):')

input5 = input('5. 조회 종료일자 입력(예:2019-01-03):')

cnt = int(input('6. 크롤링 건수 입력(최대 1000건):'))

f_name = input('7. 파일을 저장할 폴더명만 쓰세요 (예:c:\\temp\\):')

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

f_dir = 'C:\\py_temp\\test\\'

os.makedirs(f_dir+s+'-'+input1)
os.chdir(f_dir+s+'-'+input1)

ff_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.txt'
fc_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.csv'
fx_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.xls'

s_time = time.time( )

page_cnt = math.ceil(cnt / 7)

inckey = ' '.join(input2.split())
exckey = ' '.join(['-' + keyword for keyword in input3.split()])
query = input1 + ' ' + inckey + ' ' + exckey

path = "C:\py_temp\chromedriver\chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get("https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0")
time.sleep(2)

element = driver.find_element(By.NAME, "sectionBlogQuery")
element.send_keys(query)
driver.find_element(By.CLASS_NAME, 'button_blog').click()


time.sleep(2)

url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate={}&endDate={}&keyword={}".format(input4, input5,query)
driver.get(url)

time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

time.sleep(2)

r_cnt = soup.find('span', class_='search_result').find('em').get_text( )
r_cnt2 = r_cnt.replace(",", "").replace("건", "").strip()
search_cnt = int(r_cnt2)

if cnt > search_cnt :
    cnt = search_cnt
    
print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)

print("\n")

no2=[]           # 게시글 번호 컬럼
contents=[]     # 게시글 내용 컬럼
no = 1
blogUrl = []
nickname = []
date = []

for i in range(1, page_cnt+1):  
    print("%s 페이지 내용 수집 시작합니다 =======================" %i)
    for j in range(1,8):  
        f = open(ff_name, 'a',encoding='UTF-8')
        
        if no > cnt :
            break
        #각 게시글의 제목 누르기    
        time.sleep(2)
    
        elements = driver.find_elements(By.CSS_SELECTOR, '.area_list_search div:nth-child({}) div div:nth-child(1) a.desc_inner'.format(j))

        # 마지막 요소에 접근
        last_element = elements[-1]

        # href 속성 값 가져오기
        href = last_element.get_attribute('href')
        
        blogUrl.append(href)

        print(href)
        
        driver.get(href)   
        
        time.sleep(2)
    
        driver.switch_to.frame('mainFrame')
    
        html = driver.page_source
        
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            # 예외가 발생할 수 있는 코드 블록
            content_list = soup.find_all('span', 'se-fs-')
        except:
            try:
                content_list = soup.find_all('span', 'se-fs-fs16')
            except:
                try:
                    content_list = soup.find_all('span', 'se-fs-fs11')
                except:
                    print("예외 발생")
        
        tmp_nickname = soup.find('a', class_='link pcol2').text
        nickname.append(tmp_nickname)
        print('닉네임:', tmp_nickname)
    
        tmp_date = soup.find('span',class_='se_publishDate pcol2').text
        date.append(tmp_date)
        print('작성일자:', tmp_date)
        
        
        no += 1
        
        cont = ""
        for k in content_list:
            cont += k.get_text().replace("\n", "")

        contents.append(cont)
        print(cont)
        print("\n")
            
         
        
        f.write('블로그주소 :' + str(href) + '\n' + '작성자 닉네임 :' + str(tmp_nickname) + '\n' + '작성일자 :' + str(tmp_date) + '\n' + '블로그내용 :' + str(cont)+'\n\n')
        f.close( )
        no2.append(no)
        driver.back( )
        time.sleep(2)    

    if i > page_cnt+1 :
        break
        
    i += 1
    
    if (i % 7 == 0):
            driver.find_element(By.LINK_TEXT,'다음').click()
    else :
            driver.find_element(By.LINK_TEXT,'{}'.format(i)).click() # 다음 페이지번호 클릭    
    time.sleep(2)

result = pd.DataFrame()
result['번호'] = no2
result['주소'] = blogUrl
result['닉네임'] = nickname
result['작성일자'] = date
result['내용'] = contents 

# csv 형태로 저장하기
result.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
result.to_excel(fx_name,index=False)



e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)

print("=" *80)

driver.close()